In [149]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

## Data partition

In [150]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [151]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Models

In [152]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [153]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
- Fold01.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
+ Fold01.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
+ Fold01.Rep1: min.node.size=17, mtry=72, splitrule=gini 
- Fold01.Rep1: min.node.size=17, mtry=72, splitrule=gini 
+ Fold02.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
- Fold02.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
+ Fold02.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
+ Fold02.Rep1: min.node.size=17, mtry=72, splitrule=gini 
- Fold02.Rep1: min.node.size=17, mtry=72, splitrule=gini 
+ Fold03.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
- Fold03.Rep1: min.node.size= 1, mtry=59, splitrule=gini 
+ Fold03.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
- Fold03.Rep1: min.node.size= 1, mtry=28, splitrule=extratrees 
+ Fold03.Rep1: min.node.size=17, mtr

In [154]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

+ Fold01.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
- Fold01.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
+ Fold01.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
- Fold01.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
+ Fold01.Rep1: mtry=46, coefReg=0.22371, coefImp=0.9283 
- Fold01.Rep1: mtry=46, coefReg=0.22371, coefImp=0.9283 
+ Fold02.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
- Fold02.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
+ Fold02.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
- Fold02.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
+ Fold02.Rep1: mtry=46, coefReg=0.22371, coefImp=0.9283 
- Fold02.Rep1: mtry=46, coefReg=0.22371, coefImp=0.9283 
+ Fold03.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
- Fold03.Rep1: mtry=27, coefReg=0.13904, coefImp=0.4303 
+ Fold03.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
- Fold03.Rep1: mtry=42, coefReg=0.03862, coefImp=0.1208 
+ Fold03.Rep1: mtry=46, coefReg=0.22371, coefImp=0.9283 
- Fold03.Rep1: mtry=46, coefReg

## Results

In [155]:
print(model_rf)
plot(model_rf)

Random Forest 

803 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 723, 723, 722, 723, 723, 723, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
   1             28    extratrees  0.7983025  0.5967237
   1             59    gini        0.7646759  0.5295619
  17             72    gini        0.7571914  0.5145569

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 28, splitrule = extratrees
 and min.node.size = 1.


In [156]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

803 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 723, 722, 723, 723, 723, 723, ... 
Resampling results across tuning parameters:

  mtry  coefReg     coefImp    Accuracy   Kappa    
  27    0.13904262  0.4302553  0.7323920  0.4647997
  42    0.03862148  0.1207961  0.6924383  0.3849033
  46    0.22371389  0.9283021  0.7062500  0.4124489

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 27, coefReg = 0.1390426
 and coefImp = 0.4302553.


In [157]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu. Max. NA's
ranger 0.7500000 0.7678241 0.7826389 0.7983025 0.8224537  0.9    0
rrf    0.6666667 0.6966435 0.7391975 0.7323920 0.7593750  0.8    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu. Max. NA's
ranger 0.5000000 0.5363439 0.5653793 0.5967237 0.6449215  0.8    0
rrf    0.3334349 0.3936396 0.4781879 0.4647997 0.5187500  0.6    0


## Prediction

In [158]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      77      44
   NotFast   52     171
                                          
               Accuracy : 0.7209          
                 95% CI : (0.6703, 0.7677)
    No Information Rate : 0.625           
    P-Value [Acc > NIR] : 0.000112        
                                          
                  Kappa : 0.3972          
                                          
 Mcnemar's Test P-Value : 0.474959        
                                          
            Sensitivity : 0.5969          
            Specificity : 0.7953          
         Pos Pred Value : 0.6364          
         Neg Pred Value : 0.7668          
             Prevalence : 0.3750          
         Detection Rate : 0.2238          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6961          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5968992            0.7953488            0.6363636 
      Neg Pred Value            Precision               Recall 
           0.7668161            0.6363636            0.5968992 
                  F1           Prevalence       Detection Rate 
           0.6160000            0.3750000            0.2238372 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6961240

In [159]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [160]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 121 controls (TestClasses fast) < 223 cases (TestClasses NotFast).
Area under the curve: 0.7016

In [161]:
pred_rf.prob

fast,NotFast
0.278,0.722
0.124,0.876
0.888,0.112
0.070,0.930
0.686,0.314
0.178,0.822
0.636,0.364
0.612,0.388
0.242,0.758
0.170,0.830
